In [ ]:
import pandas_datareader as web
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
key=""
df = web.get_data_tiingo('AAPL', api_key='f051308453df86bec6d4eb5b51befe1f37a88e21')
df.to_csv('AAPL.csv')
df=pd.read_csv('AAPL.csv')
df

: 

In [ ]:
df=df.iloc[:,1:8]
df


: 

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD ($)',fontsize=18)
plt.show()

: 

In [ ]:
data=df.filter(['close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*.8)
training_data_len
dataset

: 

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

: 

In [ ]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if(i<=60):
        print(x_train)
        print(y_train)
        print()

: 

In [ ]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],60,1))
x_train.shape

: 

In [ ]:
#Building LSTM Model
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

: 

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

: 

In [ ]:
model.fit(x_train,y_train,batch_size=1,epochs=1)


: 

In [ ]:
test_data=scaled_data[training_data_len-60:,:]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])
x_test=np.array(x_test)
# reshaping the data
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

: 

In [ ]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)
#get the root mean squred error
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse

: 

In [ ]:
train=data[:training_data_len]
valid=data[training_data_len:]
valid['Predictions']=predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.plot(train['close'])
plt.plot(valid[['close','Predictions']])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD ($)',fontsize=18)
plt.legend(['Train','Val','Predictions'],loc='lower right')
plt.show()

: 

In [ ]:
valid

: 

: 

: 